In [1]:
import sklearn.preprocessing as preprocess
import numpy as np
import pandas as pd
from keras import backend as K

Using TensorFlow backend.


In [2]:
df_hd_tp = pd.read_csv('evaluation_set/headline_topic_mapping.csv')

In [3]:
df_hd_tp.Title.head()

0    Video game content rating system
1               Grand Theft Childhood
2                     School violence
3            Video game controversies
4               Nonviolent video game
Name: Title, dtype: object

In [8]:
import keras
from keras_bert import get_base_dict, get_model, compile_model, gen_batch_inputs


# A toy input example
sentence_pairs = [
    [['all', 'work', 'and', 'no', 'play'], ['makes', 'jack', 'a', 'dull', 'boy']],
    [['from', 'the', 'day', 'forth'], ['my', 'arm', 'changed']],
    [['and', 'a', 'voice', 'echoed'], ['power', 'give', 'me', 'more', 'power']],
]


# Build token dictionary
token_dict = get_base_dict()  # A dict that contains some special tokens
for pairs in sentence_pairs:
    for token in pairs[0] + pairs[1]:
        if token not in token_dict:
            token_dict[token] = len(token_dict)
token_list = list(token_dict.keys())  # Used for selecting a random word


# Build & train the model
model = get_model(
    token_num=len(token_dict),
    head_num=5,
    transformer_num=12,
    embed_dim=25,
    feed_forward_dim=100,
    seq_len=20,
    pos_num=20,
    dropout_rate=0.05,
)
compile_model(model)
model.summary()

def _generator():
    while True:
        yield gen_batch_inputs(
            sentence_pairs,
            token_dict,
            token_list,
            seq_len=20,
            mask_rate=0.3,
            swap_sentence_rate=1.0,
        )

model.fit_generator(
    generator=_generator(),
    steps_per_epoch=1000,
    epochs=100,
    validation_data=_generator(),
    validation_steps=100,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    ],
)


# Use the trained model
inputs, output_layer = get_model(
    token_num=len(token_dict),
    head_num=5,
    transformer_num=12,
    embed_dim=25,
    feed_forward_dim=100,
    seq_len=20,
    pos_num=20,
    dropout_rate=0.05,
    training=False,      # The input layers and output layer will be returned if `training` is `False`
    trainable=False,     # Whether the model is trainable. The default value is the same with `training`
    output_layer_num=4,  # The number of layers whose outputs will be concatenated as a single output.
                         # Only available when `training` is `False`.
)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 20)           0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 20)           0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 20, 25), (28 700         Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 20, 25)       50          Input-Segment[0][0]              
__________________________________________________________________________________________________
Embedding-

Epoch 1/100
1000/1000 [==============================] - 64s 64ms/step - loss: 3.9911 - MLM_loss: 3.3370 - NSP_loss: 0.6541 - val_loss: 3.9175 - val_MLM_loss: 3.3191 - val_NSP_loss: 0.5984
Epoch 2/100
1000/1000 [==============================] - 44s 44ms/step - loss: 3.9220 - MLM_loss: 3.2957 - NSP_loss: 0.6263 - val_loss: 3.6933 - val_MLM_loss: 3.2595 - val_NSP_loss: 0.4339
Epoch 3/100
1000/1000 [==============================] - 44s 44ms/step - loss: 3.6267 - MLM_loss: 3.2504 - NSP_loss: 0.3763 - val_loss: 3.3878 - val_MLM_loss: 3.2385 - val_NSP_loss: 0.1492
Epoch 4/100
1000/1000 [==============================] - 44s 44ms/step - loss: 3.3139 - MLM_loss: 3.2057 - NSP_loss: 0.1081 - val_loss: 3.1836 - val_MLM_loss: 3.1629 - val_NSP_loss: 0.0207
Epoch 5/100
1000/1000 [==============================] - 44s 44ms/step - loss: 3.2032 - MLM_loss: 3.1411 - NSP_loss: 0.0621 - val_loss: 3.1280 - val_MLM_loss: 3.0871 - val_NSP_loss: 0.0409
Epoch 6/100
1000/1000 [==============================] 

1000/1000 [==============================] - 44s 44ms/step - loss: 0.0355 - MLM_loss: 0.0314 - NSP_loss: 0.0041 - val_loss: 0.0125 - val_MLM_loss: 0.0117 - val_NSP_loss: 7.5070e-04
Epoch 45/100
1000/1000 [==============================] - 44s 44ms/step - loss: 0.0475 - MLM_loss: 0.0354 - NSP_loss: 0.0121 - val_loss: 0.0121 - val_MLM_loss: 0.0115 - val_NSP_loss: 6.5052e-04
Epoch 46/100
1000/1000 [==============================] - 44s 44ms/step - loss: 0.0656 - MLM_loss: 0.0528 - NSP_loss: 0.0129 - val_loss: 0.0423 - val_MLM_loss: 0.0275 - val_NSP_loss: 0.0148
Epoch 47/100
1000/1000 [==============================] - 44s 44ms/step - loss: 0.0662 - MLM_loss: 0.0526 - NSP_loss: 0.0136 - val_loss: 0.0263 - val_MLM_loss: 0.0206 - val_NSP_loss: 0.0057
Epoch 48/100
1000/1000 [==============================] - 44s 44ms/step - loss: 0.0527 - MLM_loss: 0.0413 - NSP_loss: 0.0114 - val_loss: 0.0204 - val_MLM_loss: 0.0160 - val_NSP_loss: 0.0044
Epoch 49/100
1000/1000 [==============================]

In [9]:
inputs

[<tf.Tensor 'Input-Token_1:0' shape=(?, 20) dtype=float32>,
 <tf.Tensor 'Input-Segment_1:0' shape=(?, 20) dtype=float32>]

In [10]:
output_layer

<tf.Tensor 'Encoder-Output/concat:0' shape=(?, 20, 100) dtype=float32>

In [11]:
next(_generator())

([array([[ 2,  5,  6,  7,  8,  9,  3,  4, 11, 12, 13, 14,  3,  0,  0,  0,
           0,  0,  0,  0],
         [ 2,  4,  4, 17, 18,  3,  4, 25,  4, 27, 24,  3,  0,  0,  0,  0,
           0,  0,  0,  0],
         [ 2,  7,  4,  4, 23,  3, 19,  4, 21,  3,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0]]),
  array([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
  array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])],
 [array([[[ 2],
          [ 5],
          [ 6],
          [ 7],
          [ 8],
          [ 9],
          [ 3],
          [10],
          [11],
          [12],
          [13],
          [14],
          [ 3],
          [ 0],
          [ 0],
          [ 0],
        

In [13]:
get_base_dict()

{'': 0, '[CLS]': 2, '[MASK]': 4, '[SEP]': 3, '[UNK]': 1}

In [15]:
sentence_pairs

[[['all', 'work', 'and', 'no', 'play'], ['makes', 'jack', 'a', 'dull', 'boy']],
 [['from', 'the', 'day', 'forth'], ['my', 'arm', 'changed']],
 [['and', 'a', 'voice', 'echoed'], ['power', 'give', 'me', 'more', 'power']]]

In [16]:
token_dict

{'': 0,
 '[CLS]': 2,
 '[MASK]': 4,
 '[SEP]': 3,
 '[UNK]': 1,
 'a': 12,
 'all': 5,
 'and': 7,
 'arm': 20,
 'boy': 14,
 'changed': 21,
 'day': 17,
 'dull': 13,
 'echoed': 23,
 'forth': 18,
 'from': 15,
 'give': 25,
 'jack': 11,
 'makes': 10,
 'me': 26,
 'more': 27,
 'my': 19,
 'no': 8,
 'play': 9,
 'power': 24,
 'the': 16,
 'voice': 22,
 'work': 6}

In [17]:
token_list

['',
 'no',
 'me',
 'my',
 'power',
 'echoed',
 '[UNK]',
 '[MASK]',
 'work',
 'and',
 'voice',
 'boy',
 'more',
 'arm',
 'dull',
 '[CLS]',
 '[SEP]',
 'play',
 'the',
 'a',
 'forth',
 'jack',
 'from',
 'day',
 'makes',
 'give',
 'all',
 'changed']

In [19]:
# Build token dictionary
token_dict = get_base_dict()  # A dict that contains some special tokens
for pairs in sentence_pairs:
    for token in pairs[0] + pairs[1]:
        if token not in token_dict:
            token_dict[token] = len(token_dict)

all
work
and
no
play
makes
jack
a
dull
boy
from
the
day
forth
my
arm
changed
and
a
voice
echoed
power
give
me
more
power
